# Esercizio 5 - Soluzione

### Parametri in input

In [1]:
gtf_file_name = './input.gtf'
reference_file_name = './ENm006.fa'

Scegliere una feature tra `exon` per ricostruire i trascritti e `CDS` per ricostruire le *coding sequences* del gene.

In [2]:
#selected_feature = 'exon'
selected_feature = 'CDS'

### 1) Importazione del modulo `re`

In [3]:
import re

### 2) Definizione di una funzione per estrarre da un *record* GTF il gene e il trascritto associato

La funzione prende come argomento un *record* GTF e restituisce la tupla `(gene_id, transcript_id)` contenente il gene e il trascritto associati.

### 3) Definizione di una funzione per formattare in `FASTA` una sequenza

La funzione prende come argomenti un *header* `FASTA` (che inizia con un simbolo `>`), una sequenza nucleotidica (o di proteina) e la dimensione (passata come *keyword* `record` con valore di *default* pari a 80) dei *record* in cui separare la sequenza. La funzione restituisce una stringa contenente la sequenza in formato `FASTA`.

*Suggerimento*: usare la funzione `compile()` per creare un'espressione regolare.

In [5]:
def format_fasta(header, sequence, record = 80):    
    p = re.compile('\w{,' + str(record) + '}')
    return header + '\n' + '\n'.join(re.findall(p, sequence))

### 4) Definizione di una funzione per eseguire il *reverse and complement* di una sequenza nucleotidica

La funzione prende come argomento una sequenza nucleotidica e restituisce il *reverse and complement* della sequenza in lettere maiuscole.

### 5) Definizione di una funzione che ricostruisce la sequenza di un trascritto/coding sequence

La funzione prende i seguenti argomenti:
- una lista di tuple `(start, end, ...)` che rappresentano le *features* `exon` che compongono un trascritto oppure le *features* `CDS` che compongono una *coding sequence*
- la sequenza della genomica di riferimento
- un valore di *strand* (`+/-`)

e ricostruisce la sequenza (del trascritto/CDS) tenendo conto dello strand specificato.

**NB**: se lo strand è `+` la *feature* di coordinate minori è la prima *feature* che compone la sequenza da ricostruire, altrimenti è l'ultima.

**Esempio1**: *features* `exon` che compongono la sequenza del trascritto `XX-FW83563B9.4-002` del gene `ATP6AP1`:

    ENm006	VEGA_Known	exon	542747	542902		.	+	. 
    ENm006	VEGA_Known	exon	543097	545706		.	+	.
    ENm006	VEGA_Known	exon	545879	545953		.	+	.
    ENm006	VEGA_Known	exon	546315	546508		.	+	.
    ENm006	VEGA_Known	exon	546980	547020		.	+	.
    ENm006	VEGA_Known	exon	547684	547769		.	+	.
    ENm006	VEGA_Known	exon	548257	548495		.	+	.
    
Dal momento che lo strand dei *record* è `+`, la *feature* `(542747, 542902)` di coordinate minori è il primo esone del trascritto, mentre quella di coordinate maggiori `(548257, 548495)` è l'ultimo esone del trascritto.

**Esempio2**: *features* `exon` che compongono la sequenza del trascritto `U52112.4-018` del gene `ARHGAP4`:

    ENm006	VEGA_Known	exon	79484	79511		.	-	. 
    ENm006	VEGA_Known	exon	72761	72965		.	-	.
    ENm006	VEGA_Known	exon	72521	72683		.	-	.
    ENm006	VEGA_Known	exon	72253	72379		.	-	.
    ENm006	VEGA_Known	exon	71896	71965		.	-	.

Dal momento che lo strand dei *record* è `-`, la *feature* `(79484, 79511)` di coordinate maggiori è il primo esone del trascritto, mentre quella di coordinate minori `(71896, 71965)` è l'ultimo esone del trascritto.

**Esempio3**: *features* `CDS` che compongono la sequenza della *coding sequence* del trascritto `U52112.4-019` del gene `ARHGAP4`:

    ENm006	VEGA_Known	CDS	72761	72963	.	-	0	
    ENm006	VEGA_Known	CDS	72521	72683	.	-	1	
    ENm006	VEGA_Known	CDS	72253	72315	.	-	0	
    ENm006	VEGA_Known	CDS	71872	71965	.	-	0	

Dal momento che lo strand dei *record* è `-`, la *feature* `(72761, 72963)` di coordinate maggiori è la prima *feature* della *coding sequence*, mentre quella di coordinate minori `(71872, 71965)` è l'ultima *feature* della *coding sequence*.

Per ricostruire una sequenza:

1. Ordino la lista di tuple (*features*) per coordinate crescenti
2. Concateno le sequenze delle *feature* secondo l'ordine crescente delle coordinate
3. Se lo strand è `-`, eseguo un *reverse and complement* della sequenza ottenuta

### 6) Ottenere la genomica di riferimento dal file `FASTA`

In [10]:
reference_sequence

'...'

### 7) Selezione dei *record* GTF che servono per ricostruire le sequenze del tipo scelto

In [13]:
for record in selected_gtf_records:
    print(record)

ENm006	VEGA_Known	CDS	71783	71788	.	-	0	transcript_id "U52112.4-005"; gene_id "ARHGAP4";

ENm006	VEGA_Known	CDS	70312	70440	.	-	0	transcript_id "U52112.4-005"; gene_id "ARHGAP4";

ENm006	VEGA_Known	CDS	69989	70210	.	-	0	transcript_id "U52112.4-005"; gene_id "ARHGAP4";

ENm006	VEGA_Known	CDS	64935	65036	.	-	0	transcript_id "U52112.4-005"; gene_id "ARHGAP4";

ENm006	VEGA_Known	CDS	64566	64673	.	-	0	transcript_id "U52112.4-005"; gene_id "ARHGAP4";

ENm006	VEGA_Known	CDS	64385	64459	.	-	0	transcript_id "U52112.4-005"; gene_id "ARHGAP4";

ENm006	VEGA_Known	CDS	72761	72963	.	-	0	transcript_id "U52112.4-019"; gene_id "ARHGAP4";

ENm006	VEGA_Known	CDS	72521	72683	.	-	1	transcript_id "U52112.4-019"; gene_id "ARHGAP4";

ENm006	VEGA_Known	CDS	72253	72315	.	-	0	transcript_id "U52112.4-019"; gene_id "ARHGAP4";

ENm006	VEGA_Known	CDS	71872	71965	.	-	0	transcript_id "U52112.4-019"; gene_id "ARHGAP4";

ENm006	VEGA_Known	CDS	72761	72963	.	-	0	transcript_id "U52112.4-017"; gene_id "ARHGAP4";

ENm006	VEG

### 8) Costruzione del dizionario degli *strand*

A partire dai *record* selezionati costruire il dizionario:

- *chiave*: identificatore del gene
- *valore*: strand del gene rispetto al *reference*

In [15]:
strand_dict

{'ARHGAP4': '-', 'AVPR2': '+'}

### 9) Estrazione della lista dei geni per cui è stata annotata almeno una sequenza del tipo scelto

['ARHGAP4', 'AVPR2']

### 10) Ricostruzione delle sequenze del tipo scelto

a) Costruire il dizionario degli identificatori dei trascritti:

   - *chiave*: hugo name del gene
   - *valore*: insieme degli identificatori dei trascritti per cui è annotata la sequenza da ricostruire

In [19]:
gene_dict

{'ARHGAP4': {'U52112.4-001',
  'U52112.4-003',
  'U52112.4-005',
  'U52112.4-010',
  'U52112.4-011',
  'U52112.4-017',
  'U52112.4-019',
  'U52112.4-020',
  'U52112.4-024'},
 'AVPR2': {'U52112.2-001', 'U52112.2-003'}}

b) Costruire il dizionario delle *features*:

   - *chiave*: identificatore del trascritto
   - *valore*: lista delle tuple `(start, end, frame)` delle *features* associate al trascritto che compongono la sequenza che si è scelto di ricostruire
    
**NB**: il valore di `frame` sarà *dot* (assenza di informazione) nel caso si scelga di ricostruire i trascritti.

In [21]:
composition_dict

{'U52112.4-005': [(71783, 71788, '0'),
  (70312, 70440, '0'),
  (69989, 70210, '0'),
  (64935, 65036, '0'),
  (64566, 64673, '0'),
  (64385, 64459, '0')],
 'U52112.4-019': [(72761, 72963, '0'),
  (72521, 72683, '1'),
  (72253, 72315, '0'),
  (71872, 71965, '0')],
 'U52112.4-017': [(72761, 72963, '0'),
  (72521, 72683, '1'),
  (72253, 72315, '0'),
  (71865, 71965, '0')],
 'U52112.4-010': [(63857, 63942, '2'),
  (62286, 62346, '0'),
  (61857, 61991, '2'),
  (61663, 61768, '2'),
  (61328, 61561, '1'),
  (61169, 61242, '1'),
  (60898, 61081, '2')],
 'U52112.4-020': [(72521, 72560, '1'),
  (71783, 71965, '0'),
  (70724, 70843, '0'),
  (70312, 70440, '0'),
  (70097, 70210, '0')],
 'U52112.4-003': [(77293, 77359, '0'),
  (72761, 72965, '2'),
  (72521, 72683, '1'),
  (72253, 72315, '0'),
  (71783, 71965, '0'),
  (70312, 70440, '0'),
  (69989, 70210, '0'),
  (64935, 65036, '0'),
  (64566, 64757, '0'),
  (64375, 64459, '0'),
  (64181, 64208, '2'),
  (63857, 63959, '1'),
  (62286, 62346, '0'),
  

c) Costruire un dizionario:

   - *chiave*: identificatore di trascritto
   - *valore*: *frame* della prima *feature* che compone la sequenza da ricostruire

In [23]:
frame_dict

{'U52112.4-010': '2',
 'U52112.4-001': '0',
 'U52112.4-024': '0',
 'U52112.4-017': '0',
 'U52112.4-005': '0',
 'U52112.4-020': '1',
 'U52112.4-019': '0',
 'U52112.4-003': '0',
 'U52112.4-011': '0',
 'U52112.2-003': '0',
 'U52112.2-001': '0'}

d) A partire dai primi due dizionari, ricostruire le sequenze e stamparle in un file `FASTA` con *header* simile a:

    >ARHGAP4; U52112.4-003; len=3235; type=[exon|CDS]; strand=-
    
Se si stanno ricostruendo le *coding sequences*, stampare anche in standard output la separazione in codoni tenendo conto del *frame* della prima *feature*.

>ARHGAP4; U52112.4-010; len=880; type=CDS; strand=-
Codone iniziale incompleto: GA
AAA TTC CAG AAG AGC CGC CAG CCC CGC CCC AGC TCC CAG TAT AAC CAG AGA CTC TTT GGG GGA GAC ATG GAG AAG TTT ATC CAG AGC TCA GGC CAG CCT GTG CCC CTG GTG GTG GAG AGC TGC ATT CGC TTC ATC AAC CTC AAT GGG GAG GAC CCA CTG GTG GAG GGC TGC ACT GCC CAT GAC CTG GAC TCG GTG GCC GGG GTG CTG AAG CTC TAC TTC CGG AGC CTG GAG CCC CCA CTC TTC CCC CCA GAC CTG TTC GGC GAG CTG CTG GCT TCT TCG GAG CTG GAG GCC ACA GCG GAG AGG GTG GAG CAC GTG AGC CGC CTG CTG TGG CGG CTG CCC GCG CCG GTG CTG GTG GTT CTG CGC TAC CTC TTC ACC TTC CTC AAC CAC CTG GCC CAG TAC AGC GAT GAG AAC ATG ATG GAC CCC TAC AAC CTG GCC GTG TGC TTC GGG CCC ACG CTG CTA CCG GTG CCC GCT GGG CAG GAC CCG GTG GCG CTG CAG GGC CGG GTG AAC CAG CTG GTG CAG ACG CTC ATA GTG CAG CCC GAT CGG GTC TTC CCG CCC CTG ACC TCG CTG CCT GGC CCC GTC TAC GAG AAG TGC ATG GCA CCG CCT TCC GCC AGC TGC CTG GGG GAC GCC CAG CTG GAG AGC CTG GGG GCG GAC AAT GAG CCG GAG CTG GAA GCC GAG ATG CCC GCA CAG G